In [ ]:
import pandas as pd
import numpy as np
from rapidfuzz import process, utils, fuzz

In [ ]:
data=pd.read_csv('C:/wamp/tmp/CSVS_2/recommendations_master_sheet_1 - Copy.csv')
df=pd.DataFrame(data)
required_cols=['id','age','location','gender','reason','source_type','skin_type','product_type','product_name']
#df[required_cols].tail(10)

In [ ]:
sorted_df = df.sort_values(by="product_name")
sorted_df.fillna('', inplace=True)

In [ ]:
sorted_df_list = sorted_df.values.tolist()
sorted_df = sorted_df.reset_index(drop=True)
unique_product_names = sorted_df['product_name'].unique().tolist()
#unique_product_names

In [ ]:
all_sorted_df_list = sorted_df['product_name'].tolist()
all_sorted_df_list

In [ ]:
processed_products = [utils.default_process(product) for product in all_sorted_df_list]
processed_products

In [ ]:
from rapidfuzz.process import cdist

def find_matched_product():
    # Calculate distance between all the names
    sa = cdist(processed_products, processed_products,score_cutoff=80, workers=-1)
    matches_list = []
    for distances in sa:
        indices = np.argwhere(~np.isin(distances, [100, 0])).flatten()
        names = list(map(processed_products.__getitem__, indices))
        matches_list.append(names)
    dfff = pd.DataFrame({'second_df_names_of_product': processed_products, 'matches': matches_list})
    dfff['matches_count'] = dfff.matches.str.len()
    return dfff

In [ ]:
dfff=find_matched_product()

In [ ]:
df_merge = pd.concat([sorted_df,dfff], axis=1)

In [ ]:
#df_merge.head(70)

In [ ]:
best_matches=[]
for ind in df_merge.index:
    matches_for_product_list=df_merge['matches'][ind]
    if len(matches_for_product_list)==0:
        best_matches.append(df_merge['second_df_names_of_product'][ind])
    else:
        max_len_match = get_max_length_match(matches_for_product_list)
        best_matches.append(max_len_match)
df_merge['best_match']=best_matches

In [ ]:
def get_max_length_match(match):
    max_len=0
    max_len_item=""
    for item in match:
        if(len(item)>max_len):
            max_len=len(item)
            max_len_item=item
    return max_len_item

In [ ]:
df_merge.head(80)